# Strutture dati e controllo

Un elemento fondamentale nella progettazione di software efficiente, o di algoritmi non banali, è la disponibilità di strutture dati che consentano di rappresentare i dati in maniera molto semplice e naturale. A loro volta, le strutture dati devono supportare le operazioni di modifica e accesso agli elementi in maniera efficiente oltre che semplice.  
Dati e controllo sono quindi legati in maniera molto stretta e ciò si riflette nella corrispondente disponibilità di meccanismi programmativi adeguati per questi scopi.

In python gli ```iteratori``` e ```iterabili``` sono un esempio notevole di questi meccanismi che prenderemo in considerazione in questo notebook. Oltre ad essi, studieremo i ```generatori```, meccanismi molto più affini al concetto di funzione ma che implementa lo stesso meccanismo degli iteratori. Per concludere parleremo anche di ```coroutine```. Queste possono essere viste in python come un'estensione dei generatori, ma che implementa un protocollo più bidirezionale, ovvero tra chiamante e chiamato.

## Struttura iterabile

Un iterabile in python, come dice già la parole in sè, è una struttura su cui si può iterare, ovvero accedere a tutti gli elementi secondo un ordine prestabilito. Più semplicemente, un iteratore è un contenitore ordinato di oggetti generati all'occorrenza, ovvero quando viene richiesto l'oggetto successivo viene calcolato.

Vediamo subito un esempio di iteratore che abbiamo usato senza saperlo.

In [5]:
# Iterabile che permette ad ogni chiamata di 
# Restituire l'oggetto successivo, quindi
# 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
R = range(1, 10)
print(type(R))

<class 'range'>


L'iterabile range definisce quindi un intervallo di interi. Esso è definito da tre numeri, l'estremo iniziale, finale e il passo da fare ogni iterazione. Se il passo non viene specificato, lo si assume pari a 1.

In Python 2 dichiarare un range era analogo a dichiarare una lista contenete i valori che avrebbe restituito il range. Questa implementazione di range non utilizzava il concetto di iterabile, infatti come grande svantaggio aveva che su grossi intervalli venivano allocati tutti i numeri dell'intervallo stesso, e quindi un grosso utilizzo di memoria solamente per ottenere un insieme di numeri.  
Python 3 introdusse il concetto di iterabile e range ebbe subito un notevole beneficio. Come prima cosa, range non restituiva una lista di interi, ma un oggetto range, come visto sopra. Questo oggetto iterabile ha la capacità di creare l'intero successivo all'occorrenza seguendo determinate regole, nel caso del range il passo. Facendo questo cambiamento implementativo in range, quando si richiede un grosso insieme di numeri, la memoria non viene più riempita come nella precedente versione, e quindi ne beneficia sia essa che le prestazioni.

Questa spiegazione di iterabile segue immediatamente il concetto di iteratore.  
L'iteratore è del codice che sa come accedere, generare, uno ad uno gli elementi dell'iterabile.

Vediamo un esempio di iterabile e iteratore con range.

In [8]:
iterabile = range(10)
iteratore = iter(iterabile)

print(type(iterabile))
print(type(iteratore))

<class 'range'>
<class 'range_iterator'>


In [9]:
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))

0
1
2
3


In [10]:
iterabile = "Python 3"
iteratore = iter(iterabile)

print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))

P
y
t
h


In [12]:
iterabile = [1, "a", (2, "b"), True]
iteratore = iter(iterabile)

print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))

1
a
(2, 'b')
True


Quindi, la sostanziale differenza tra iterabile e iteratore è la seguente:

- Iterabile, contenitore di oggetti sul quale è possibile iterare
- Iteratore, codice che conosce il come iterare l'iteratore

Chiaramente l'iteratore deve implementare un meccanismo per quando l'iterabile è concluso. Nel caso si arrivi alla conclusione dell'iterabile, l'iteratore solleverà un'eccezzione ```StopIteration```.

In [14]:
iterabile = range(3)
iteratore = iter(iterabile)

print(next(iteratore))
print(next(iteratore))
print(next(iteratore))
print(next(iteratore))      # Solleva eccezzione StopIteration

0
1
2


StopIteration: 

In [15]:
iterabile = range(3)
iteratore = iter(iterabile)

while True:
    try:
        elemento = next(iteratore)
    except StopIteration:
        break
    else:
        print(elemento)

0
1
2


Abbiamo detto nello scorso notebook che il for in python è molto diverso di for di altri linguaggi. L'iterabile e l'iteratore ne è il motivo.  
Il for richiede un oggetto iterabile per essere utilizzato e quello che fa il for sostanzialmente è chiamare l'oggetto successivo dall'iterabile e gestire in maniera automatica la conclusione dell'iterabile. Sostanzialmente il for è l'implementazione con il while e il try/except appena vista sopra.

In [16]:
iterabile = range(3)

for elemento in iterabile:
    print(elemento)

0
1
2


## Costruzione di strutture

Abbiamo visto che gli iteratori consentano di percorrere una struttura, ora vediamo come costruire una struttura efficiente.

Immaginiamo di avere bisogno di una lista dei quadrati dei primi n numeri a partire da 0. Potremmo usare un for e le liste.

In [19]:
def squares(n):
    squares_list = []
    for number in range(n):
        squares_list.append(number ** 2)
    return squares_list

l = squares(10)
print(l)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


Ma chiaramente questa è un'implementazione analoga al range in python 2. Questa implementazione genera tutti i valori e non c'è altro modo che generarli tutti.

Una seconda possibilità per risolvere il nostro problema è di crearci una struttura dati iterabile ad hoc.

In [25]:
class Squares:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i == self.n:
            raise StopIteration
        e = self.i
        self.i += 1
        return e ** 2

In [27]:
s = Squares(10)

print(type(s))
print(s)

for e in s:
    print(e)

<class '__main__.Squares'>
0
1
4
9
16
25
36
49
64
81


## List comprehension

La list comprehension è un meccanismo che rende semplice e compatto la costruzione di una lista e non solo, vale anche per altre strutture dati.  

Immaginiamo di avere un file di log contenente una serie di indirizzi. Quello che vogliamo è avere una lista in python contenente solamente gli indirizzi appartenenti a UNIMORE, quindi di dominio ```unimore.it```.

Una soluzione classica può essere la seguente.

In [2]:
unimore = []

with open("files/addresses.log", "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.find("unimore.it") != -1:
            unimore.append(line.strip())

print("UNIMORE Domains:")
for domain in unimore:
    print(domain)

UNIMORE Domains:
unimore.it
www.ingmo.unimore.it
inginf.unimore.it


Come si può notare dall'esempio, ci sono molte righe di codice per un compito semplice. Qui ci aiuta python con la list comprehension.

Vediamo l'esempio e poi lo commentiamo.

In [8]:
unimore = []

with open("files/addresses.log", "r") as file:
    lines = file.readlines()
    unimore = [line.strip() for line in lines if line.find("unimore.it") != -1]

print(type(unimore))
print("UNIMORE Domains:")
for domain in unimore:
    print(domain)

<class 'list'>
UNIMORE Domains:
unimore.it
www.ingmo.unimore.it
inginf.unimore.it


Il codice qui sopra potrebbe essere ridotto ulteriormente, ma ne andrebbe a discapito della facilità di lettura del codice stesso.

Quindi la list comprehension ci permette di creare una struttura dati in una unica linea di codice mettendo anche delle condizioni da rispettare.

La scrittura generica è la seguente.

```python
[<espressione> for <id> in <iterabile> if <condizione>]
```

Vediamo ora un esempio di list comprehension usando tuple e dizionari.

In [9]:
unimore = ()

with open("files/addresses.log", "r") as file:
    lines = file.readlines()
    unimore = (line.strip() for line in lines if line.find("unimore.it") != -1)

print(type(unimore))
print("UNIMORE Domains:")
for domain in unimore:
    print(domain)

<class 'generator'>
UNIMORE Domains:
unimore.it
www.ingmo.unimore.it
inginf.unimore.it


In [12]:
unimore = {}

with open("files/addresses.log", "r") as file:
    lines = file.readlines()
    unimore = {index: line.strip() for index, line in zip(range(10), lines) if line.find("unimore.it") != -1}

print(type(unimore))
print("UNIMORE Domains:")
for index, domain in unimore.items():
    print(index, domain)

<class 'dict'>
UNIMORE Domains:
2 unimore.it
4 www.ingmo.unimore.it
5 inginf.unimore.it


Notiamo qualcosa di particolare nella list comprehension della tupla. Notiamo che il tipo che ritorna è un ```generator```. Quindi è arrivato il momento di analizzare cosa siano i generatori.

## Generatori

Un generatore implementa lo stesso protocollo di un iteratore, ma è definibile in modo più semplice a partire dalla ben nota nozione di funzione.  
Con il termine generatore solitamente racchiudiamo due concetti differenti:

- Funzione generatore
- Oggetto generatore

La funzione generatore è la funzione in sè, la funzione che implementa la logica dell'iterazione dei valori.  
L'oggetto generatore è invece l'oggetto che permette di iterare e sul quale è possibile chiamare il metodo ```next```.

Strutturalmente, una funzione viene considerata generatore se implementa all'interno della definizione l'istruzione ```yeld``` al posto della classica ```return```. L'istruzione yeld permette di ritornare un determinato parametro ogni volta che viene utilizzato next.

Vediamo un possibile generatore.

In [13]:
# Funzione generatore
def fibonacci():
    a = 0
    b = 1
    while True:
        yield b
        a, b = b, a + b

In [14]:
# Oggetto generatores
f = fibonacci()

print(type(f))

<class 'generator'>


In [15]:
for i in range(3):
    print(next(f))

1
1
2


Quando viene chiamato il primo next, python esegue la funzione generatore fino a quando non raggiunge l'istruzione yield. A quel punto, l'esecuzione della funzione si interrompe e viene ritornato al metodo next il valore in yield.  
Alla chiamata successiva di next, python fa ripartire l'esecuzione della funzione del punto di interruzzione precedente, quindi riprende a eseguire dalla yield della chiamataa precedente fino ad arrivare nuovamente alla yield corrente.

Torniamo ora un attimo alla list comprehension sulla tupla. Abbiamo notato come generasse un generatore. Ora che abbiamo capito cosa sia, possiamo usufruirne con la next.  
Quindi la list comprehension sulle tuple è sostanzialmente un metodo veloce e rapido, anche se limitato, di creare dei generatori inline.

## Coroutine

La coroutine è sostanzialmente un generatore bidirezionale. Infatti il generatore non è in grado di ricevere input dato che la chiatata su di esso viene effettuata dalla next.  
Detto ciò, la yield in realtà è un'istruzione bidirezionale, ovvero è in grado di reicevere e restituire un valore. Il suo uso generale è il seguente:

```python
in = yield out
```

Dove ```in``` è una variabile e ```out``` è un'espressione. Importante è l'ordine di esecuzione. Prima il generatore restituisce un valore al chiamante e solo successivamente il generatore viene riattivato ricevendo un input dal chiamate. Per inviare un valore al generatore si usa l'istruzione ```send```.

Quindi un generatore al quale si può passare input è detta coroutine. Vediamo un esempio semplice.

In [16]:
def coroutine():
    print("Coroutine started")
    for i in range(1, 6):
        x = (yield i ** 2)
        print(f"Message to coroutine: {x}")

c = coroutine()

print("Coroutine created")
print(f"Message from coroutine: {next(c)}")

try:
    while True:
        x = input("Enter a value: ")
        print(f"Message from coroutine: {c.send(x)}")
except StopIteration:
    print("Coroutine ended")

Coroutine created
Coroutine started
Message from coroutine: 1
Message to coroutine: 1
Message from coroutine: 4
Message to coroutine: 2
Message from coroutine: 9
Message to coroutine: 3
Message from coroutine: 16
Message to coroutine: 2
Message from coroutine: 25
Message to coroutine: 
Coroutine ended


Attenzione, è molto importante che una coroutine parta con un next, perchè le coroutine hanno la necessità prima di mandare informazioni e solamente dopo riceverle, proprio dovuto al meccanismo dell'istruzione yield.